## Use a serverless Spark compute

You should have an attached Synapse Spark pool available in your workspace. Please see documentation page: [Attach and manage a Synapse Spark pool in Azure Machine Learning (preview)](https://learn.microsoft.com/azure/machine-learning/how-to-manage-synapse-spark-pool) for more details.

**Note** - To ensure successful execution of Spark job, the identity being used for the Spark job should be assigned **Contributor** and **Storage Blob Data Contributor** roles on the Azure storage account used for data input and output.

In [ ]:
from azure.ai.ml import MLClient, spark
from azure.identity import DefaultAzureCredential
import time

subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

spark_job = spark(
    display_name="interactive_data_wrangling",
    code="../../../data-wrangling",
    entry={"file": "interactive_data_wrangling.py"},
    driver_cores=1,
    driver_memory="2g",
    executor_cores=2,
    executor_memory="2g",
    executor_instances=2,
    resources={
        "instance_type": "Standard_E8S_V3",
        "runtime_version": "3.4.0",
    },
)

# Create or update the Spark job
returned_spark_job = ml_client.jobs.create_or_update(spark_job)

print(returned_spark_job.id)
# Wait until the job completes
job_name = returned_spark_job.name
while True:
    job = ml_client.jobs.get(job_name)
    print(f"Current job status: {job.status}")
    if job.status in ["Completed", "Failed", "Canceled"]:
        break
    time.sleep(10)

print(f"Final job status: {job.status}")